In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import timezone
from scipy.signal import find_peaks

plt.style.use('ggplot')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/Projects/Ian_SensAI/data/SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv'
df = pd.read_csv(path)
df.shape

(465885, 11)

Step 2 - Uniform Sampling for Current, Voltage, Temperature

For the times series data, convert the rows D, E, K  to uniform time step (Uniform sampling). These values should be easily changed, but we can start with 10 seconds.
We will be doing this for the Current, Voltage, and Temperature time series that are in rows D, E and K of the times series files.
Lastly, from the uniform sampled data, create an additional time series for Power which equals Current x Voltage.

In [ ]:
df.head(10)

,Date_Time,Test_Time (s),Cycle_Index,Current (A),Voltage (V),Charge_Capacity (Ah),Discharge_Capacity (Ah),Charge_Energy (Wh),Discharge_Energy (Wh),Environment_Temperature (C),Cell_Temperature (C)
0,2018-02-01 10:02:10.002962,10.002,1.0,0.000,2.940,0.000,0.0,0.000,0.0,15.163,15.375
1,2018-02-01 10:02:20.016433,20.016,1.0,0.000,2.941,0.000,0.0,0.000,0.0,15.148,15.437
2,2018-02-01 10:02:30.031429,30.031,1.0,0.000,2.941,0.000,0.0,0.000,0.0,15.087,15.375
3,2018-02-01 10:02:40.046426,40.046,1.0,0.000,2.941,0.000,0.0,0.000,0.0,15.240,15.375
4,2018-02-01 10:02:50.061462,50.061,1.0,0.000,2.941,0.000,0.0,0.000,0.0,15.087,15.375
5,2018-02-01 10:03:00.076593,60.076,1.0,0.000,2.941,0.000,0.0,0.000,0.0,15.163,15.375
6,2018-02-01 10:03:00.170421,60.170,1.0,0.549,2.995,0.000,0.0,0.000,0.0,15.072,15.437
7,2018-02-01 10:03:02.728858,62.728,1.0,0.549,3.006,0.000,0.0,0.001,0.0,15.087,15.375
8,2018-02-01 10:03:07.252841,67.252,1.0,0.549,3.016,0.001,0.0,0.003,0.0,15.087,15.375
9,2018-02-01 10:03:12.712738,72.712,1.0,0.549,3.026,0.001,0.0,0.005,0.0,15.010,15.298


In [ ]:
CURRENT = 'Current (A)'
VOLTAGE = 'Voltage (V)'
TEMP    = 'Cell_Temperature (C)'
DATE = 'Date_Time'

In [ ]:
df = df[[DATE, TEMP, CURRENT, VOLTAGE]].copy()

In [ ]:
df['Date_Time'] = pd.to_datetime(df['Date_Time'])

df['Date_Time'] = df['Date_Time'].dt.round(freq='T') # minutely frequency

In [ ]:
df.head(10)

,Date_Time,Cell_Temperature (C),Current (A),Voltage (V)
0,2018-02-01 10:02:00,15.375,0.000,2.940
1,2018-02-01 10:02:00,15.437,0.000,2.941
2,2018-02-01 10:03:00,15.375,0.000,2.941
3,2018-02-01 10:03:00,15.375,0.000,2.941
4,2018-02-01 10:03:00,15.375,0.000,2.941
5,2018-02-01 10:03:00,15.375,0.000,2.941
6,2018-02-01 10:03:00,15.437,0.549,2.995
7,2018-02-01 10:03:00,15.375,0.549,3.006
8,2018-02-01 10:03:00,15.375,0.549,3.016
9,2018-02-01 10:03:00,15.298,0.549,3.026


In [ ]:
print(df.shape)
df.drop_duplicates(subset=[DATE], keep='last', inplace=True)
print(df.shape)

(465885, 4)
(399672, 4)


In [ ]:
df.head(10)

,Date_Time,Cell_Temperature (C),Current (A),Voltage (V)
1,2018-02-01 10:02:00,15.437,0.000,2.941
11,2018-02-01 10:03:00,15.375,0.549,3.047
18,2018-02-01 10:04:00,15.360,0.549,3.117
24,2018-02-01 10:05:00,15.298,0.549,3.164
29,2018-02-01 10:06:00,15.375,0.549,3.196
35,2018-02-01 10:07:00,15.298,0.550,3.228
41,2018-02-01 10:08:00,15.236,0.549,3.255
47,2018-02-01 10:09:00,15.236,0.549,3.278
53,2018-02-01 10:10:00,15.344,0.549,3.293
59,2018-02-01 10:11:00,15.360,0.549,3.299


In [ ]:
timestamp_df = df.copy().reset_index(drop=True)
timestamp_df['Date_Time'] = timestamp_df['Date_Time'].apply(lambda x: x.timestamp())

In [ ]:
timestamp_df.head()

,Date_Time,Cell_Temperature (C),Current (A),Voltage (V)
0,1.517479e+09,15.437,0.000,2.941
1,1.517479e+09,15.375,0.549,3.047
2,1.517479e+09,15.360,0.549,3.117
3,1.517480e+09,15.298,0.549,3.164
4,1.517480e+09,15.375,0.549,3.196


In [ ]:
timestamp_df.tail()

,Date_Time,Cell_Temperature (C),Current (A),Voltage (V)
399667,1.581791e+09,15.564,0.0,2.613
399668,1.581791e+09,15.549,0.0,2.624
399669,1.581792e+09,15.487,0.0,2.633
399670,1.581792e+09,15.472,0.0,2.647
399671,1.581792e+09,15.349,0.0,2.648


In [ ]:
interval = 10

first_date_time = timestamp_df['Date_Time'].iloc[0]
last_date_time = timestamp_df['Date_Time'].iloc[-1] + interval

first_date_time, last_date_time

(1517479320.0, 1581791650.0)

In [ ]:
devided_values = np.arange(first_date_time ,last_date_time, interval)
len(devided_values)

6431233

In [ ]:
a_dataframe = pd.DataFrame(devided_values, columns= ['Time'])
len(a_dataframe)

6431233

In [ ]:
final_df = pd.merge(a_dataframe, timestamp_df, how='left', left_on='Time',right_on='Date_Time')
len(final_df)

6431233

In [ ]:
final_df.head(10)

,Time,Date_Time,Cell_Temperature (C),Current (A),Voltage (V)
0,1.517479e+09,1.517479e+09,15.437,0.000,2.941
1,1.517479e+09,NaN,NaN,NaN,NaN
2,1.517479e+09,NaN,NaN,NaN,NaN
3,1.517479e+09,NaN,NaN,NaN,NaN
4,1.517479e+09,NaN,NaN,NaN,NaN
5,1.517479e+09,NaN,NaN,NaN,NaN
6,1.517479e+09,1.517479e+09,15.375,0.549,3.047
7,1.517479e+09,NaN,NaN,NaN,NaN
8,1.517479e+09,NaN,NaN,NaN,NaN
9,1.517479e+09,NaN,NaN,NaN,NaN


In [ ]:
x = (timestamp_df['Date_Time'].to_numpy()).astype(float)
y_curreny = (timestamp_df[CURRENT].to_numpy()).astype(float)
y_voltage = (timestamp_df[VOLTAGE].to_numpy()).astype(float)
y_temp    = (timestamp_df[TEMP].to_numpy()).astype(float)

In [ ]:
result_current = np.interp(devided_values, x, y_curreny)
result_voltage = np.interp(devided_values, x, y_voltage)
result_temp = np.interp(devided_values, x, y_temp)

len(result_current)

6431233

In [ ]:
final_df[CURRENT] =  result_current
final_df[VOLTAGE] =  result_voltage
final_df[TEMP]    =  result_temp 
final_df['Power'] = final_df[CURRENT] * final_df[VOLTAGE]

In [ ]:
final_df

,Time,Date_Time,Cell_Temperature (C),Current (A),Voltage (V),Power
0,1.517479e+09,1.517479e+09,15.437000,0.0000,2.941000,0.000000
1,1.517479e+09,NaN,15.426667,0.0915,2.958667,0.270718
2,1.517479e+09,NaN,15.416333,0.1830,2.976333,0.544669
3,1.517479e+09,NaN,15.406000,0.2745,2.994000,0.821853
4,1.517479e+09,NaN,15.395667,0.3660,3.011667,1.102270
...,...,...,...,...,...,...
6431228,1.581792e+09,NaN,15.431000,0.0000,2.647333,0.000000
6431229,1.581792e+09,NaN,15.410500,0.0000,2.647500,0.000000
6431230,1.581792e+09,NaN,15.390000,0.0000,2.647667,0.000000
6431231,1.581792e+09,NaN,15.369500,0.0000,2.647833,0.000000


In [ ]:
final_df.drop(columns=['Date_Time'], inplace=True) 

In [ ]:
final_df['Time'] = pd.to_datetime(final_df['Time'], unit='s')

In [ ]:
final_df.sample(5)

,Time,Cell_Temperature (C),Current (A),Voltage (V),Power
4378268,2019-06-23 03:53:20,15.124467,0.042467,3.588000,0.152370
1847534,2018-09-03 06:04:20,16.704000,0.549000,3.197667,1.755519
3788956,2019-04-15 22:54:40,15.588000,0.549000,3.364000,1.846836
971920,2018-05-24 21:48:40,15.717167,0.549000,3.380833,1.856077
3020577,2019-01-17 00:31:30,17.090858,-1.050350,3.093267,-3.249012


In [ ]:
final_df.to_csv('SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv', index=False)

In [ ]:
! zip -9 SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.zip /content/SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv

  adding: content/SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv (deflated 79%)


In [ ]:
! cp SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.zip /content/drive/MyDrive/Projects/Ian_SensAI/data/